In [2]:
import json
import nltk
from nltk.corpus import stopwords
import re
import networkx
from textblob import TextBlob
import pandas as pd
import numpy as np
from scipy import spatial
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import os
import tweepy as tw
import pandas as pd

from wordcloud import WordCloud 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


Sentiment= pd.read_csv("SAD.csv", error_bad_lines = False)



b'Skipping line 8836: expected 4 fields, saw 5\n'
b'Skipping line 535882: expected 4 fields, saw 7\n'


In [3]:
Sentiment.head()

,ItemID,Sentiment,SentimentSource,SentimentText
0,1,0,Sentiment140,is so sad for my APL frie...
1,2,0,Sentiment140,I missed the New Moon trail...
2,3,1,Sentiment140,omg its already 7:30 :O
3,4,0,Sentiment140,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,Sentiment140,i think mi bf is cheating on me!!! ...


In [4]:
len(Sentiment)

1578612

In [5]:
Sentiment.Sentiment.value_counts()

1    790177
0    788435
Name: Sentiment, dtype: int64

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


tvec = TfidfVectorizer(stop_words = None, max_features = 100000, ngram_range = (1,3))
lr = LogisticRegression()


def LogisticRegression_kfoldcv(splits, X, Y, pipeline, average_method):
    
    kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=777)
    accuracy = []
    precision = []
    recall = []
    f1 = []
    for train, test in kfold.split(X, Y):
        lr_fit = pipeline.fit(X[train], Y[train])
        prediction = lr_fit.predict(X[test])
        scores = lr_fit.score(X[test],Y[test])
        
        accuracy.append(scores * 100)
        precision.append(precision_score(Y[test], prediction, average=average_method)*100)
        print('             negative     positive')
        print('precision:',precision_score(Y[test], prediction, average=None))
        recall.append(recall_score(Y[test], prediction, average=average_method)*100)
        print('recall:   ',recall_score(Y[test], prediction, average=None))
        f1.append(f1_score(Y[test], prediction, average=average_method)*100)
        print('f1 score: ',f1_score(Y[test], prediction, average=None))
        print('-'*50)

    print("accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
    print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
    print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
    print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))

In [7]:
from sklearn.pipeline import Pipeline

original_pipeline = Pipeline([
    ('vectorizer', tvec),
    ('classifier', lr)
])

In [20]:
LogisticRegression_kfoldcv(5, Sentiment.SentimentText, Sentiment.Sentiment, original_pipeline, 'macro') 


             negative     positive
precision: [0.82698251 0.81513863]
recall:    [0.81120194 0.83065884]
f1 score:  [0.81901622 0.82282555]
--------------------------------------------------
             negative     positive
precision: [0.82944874 0.81730656]
recall:    [0.81335811 0.83312663]
f1 score:  [0.82132462 0.82514077]
--------------------------------------------------
             negative     positive
precision: [0.82814961 0.81473749]
recall:    [0.81034581 0.83221438]
f1 score:  [0.81915098 0.82338321]
--------------------------------------------------
             negative     positive
precision: [0.82746305 0.8140137 ]
recall:    [0.80958481 0.83156263]
f1 score:  [0.81842631 0.82269459]
--------------------------------------------------
             negative     positive
precision: [0.82906586 0.81674548]
recall:    [0.81273028 0.83280286]
f1 score:  [0.8208168  0.82469602]
--------------------------------------------------
accuracy: 82.18% (+/- 0.11%)
precision: 82.19

In [3]:
Sentiment.head()

,ItemID,Sentiment,SentimentSource,SentimentText
0,1,0,Sentiment140,is so sad for my APL frie...
1,2,0,Sentiment140,I missed the New Moon trail...
2,3,1,Sentiment140,omg its already 7:30 :O
3,4,0,Sentiment140,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,Sentiment140,i think mi bf is cheating on me!!! ...


In [ ]:
len(Sentiment)

In [13]:
from sklearn.naive_bayes import MultinomialNB

In [46]:
#Naive Bayes Classifier

X_train, X_test, y_train, y_test = train_test_split(Sentiment['SentimentText'][:20000], Sentiment['Sentiment'][:20000], test_size=0.2)
pipeline = Pipeline([
    ('bow', CountVectorizer(strip_accents='ascii',
                            stop_words='english',
                            lowercase=True)), 
    ('tfidf', TfidfTransformer()),  
    ('classifier', MultinomialNB()),
])

parameters = {'bow__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'classifier__alpha': (1e-2, 1e-3),
             }

grid = GridSearchCV(pipeline, cv=10, param_grid=parameters, verbose=1)
grid.fit(X_train,y_train)


print(grid.best_score_, grid.best_params_)
print('\n')
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(mean, stdev, param)

Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:   42.7s finished



Best Model: 0.705187 using {'bow__ngram_range': (1, 1), 'classifier__alpha': 0.01, 'tfidf__use_idf': False}


Mean: 0.689625 Stdev:(0.014710) with: {'bow__ngram_range': (1, 1), 'classifier__alpha': 0.01, 'tfidf__use_idf': True}
Mean: 0.705187 Stdev:(0.014644) with: {'bow__ngram_range': (1, 1), 'classifier__alpha': 0.01, 'tfidf__use_idf': False}
Mean: 0.685125 Stdev:(0.015297) with: {'bow__ngram_range': (1, 1), 'classifier__alpha': 0.001, 'tfidf__use_idf': True}
Mean: 0.694688 Stdev:(0.015252) with: {'bow__ngram_range': (1, 1), 'classifier__alpha': 0.001, 'tfidf__use_idf': False}
Mean: 0.689500 Stdev:(0.009140) with: {'bow__ngram_range': (1, 2), 'classifier__alpha': 0.01, 'tfidf__use_idf': True}
Mean: 0.704125 Stdev:(0.007669) with: {'bow__ngram_range': (1, 2), 'classifier__alpha': 0.01, 'tfidf__use_idf': False}
Mean: 0.681625 Stdev:(0.008409) with: {'bow__ngram_range': (1, 2), 'classifier__alpha': 0.001, 'tfidf__use_idf': True}
Mean: 0.689500 Stdev:(0.006624) with: {'bow__ngram_range'

In [24]:
from sklearn.externals import joblib

In [47]:
joblib.dump(grid, "twitter_sentiment3.pkl")

['twitter_sentiment3.pkl']

In [41]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [48]:
model_NB = joblib.load("twitter_sentiment3.pkl" )

y_preds = model_NB.predict(X_test)

print('accuracy score: ',accuracy_score(y_test, y_preds))
print('\n')
print('confusion matrix: \n',confusion_matrix(y_test,y_preds))
print('\n')
print( )


accuracy score:  0.7195


confusion matrix: 
 [[1703  474]
 [ 648 1175]]


              precision    recall  f1-score   support

           0       0.72      0.78      0.75      2177
           1       0.71      0.64      0.68      1823

    accuracy                           0.72      4000
   macro avg       0.72      0.71      0.71      4000
weighted avg       0.72      0.72      0.72      4000



In [132]:
x1 = tvec.fit_transform(Sentiment['SentimentText'].values.astype('U'))
y = Sentiment.Sentiment

In [80]:
from sklearn.model_selection import train_test_split
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.02, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)


from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from time import time



In [81]:
def accuracy(pipeline, x_train, y_train, x_test, y_test):
    if len(x_test[y_test == 0]) / (len(x_test)*1.) > 0.5:
        null_accuracy = len(x_test[y_test == 0]) / (len(x_test)*1.)
    else:
        null_accuracy = 1. - (len(x_test[y_test == 0]) / (len(x_test)*1.))
    t0 = time()
    sentiment_fit = pipeline.fit(x_train, y_train)
    y_pred = sentiment_fit.predict(x_test)
    train_test_time = time() - t0
    accuracy = accuracy_score(y_test, y_pred)
    print ("null accuracy: {0:.2f}%".format(null_accuracy*100))
    print ("accuracy score: {0:.2f}%".format(accuracy*100))
    if accuracy > null_accuracy:
        print ("model is {0:.2f}% more accurate than null accuracy".format((accuracy-null_accuracy)*100))
    elif accuracy == null_accuracy:
        print ("model has the same accuracy with the null accuracy")
    else:
        print ("model is {0:.2f}% less accurate than null accuracy".format((null_accuracy-accuracy)*100))
    print ("train and test time: {0:.2f}s".format(train_test_time))
    print ("-"*80)
    return accuracy, train_test_time

In [82]:
cvec = CountVectorizer()
lr = LogisticRegression()
n_features = np.arange(10000,100001,10000)

def accuracy2(vectorizer=cvec, n_features=n_features, stop_words=None, ngram_range=(1, 1), classifier=lr):
    result = []
    print (classifier)
    print ("\n")
    for n in n_features:
        vectorizer.set_params(stop_words=stop_words, max_features=n, ngram_range=ngram_range)
        pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', classifier)
        ])
        print ("Validation for {} features".format(n))
        nfeature_accuracy,tt_time = accuracy(pipeline, x_train, y_train, x_validation, y_validation)
        result.append((n,nfeature_accuracy,tt_time))
    return result

In [83]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.neighbors import NearestCentroid
from sklearn.feature_selection import SelectFromModel

names = ["Logistic Regression", "Linear SVC", "LinearSVC with L1-based feature selection","Multinomial NB", 
         "Bernoulli NB", "Ridge Classifier", "AdaBoost", "Perceptron","Passive-Aggresive", "Nearest Centroid"]
classifiers = [
    LogisticRegression(),
    LinearSVC(),
    Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False))),
  ('classification', LinearSVC(penalty="l2"))]),
    MultinomialNB(),
    BernoulliNB(),
    RidgeClassifier(),
    AdaBoostClassifier(),
    Perceptron(),
    PassiveAggressiveClassifier(),
    NearestCentroid()
    ]
zipped_clf = zip(names,classifiers)

In [84]:
tvec = TfidfVectorizer()
def classifier(vectorizer=tvec, n_features=10000, stop_words=None, ngram_range=(1, 1), classifier=zipped_clf):
    result = []
    vectorizer.set_params(stop_words=stop_words, max_features=n_features, ngram_range=ngram_range)
    for n,c in classifier:
        checker_pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', c)
        ])
        print ("Validation result for {}".format(n))
        print (c)
        clf_accuracy,tt_time = accuracy2(checker_pipeline, x_train, y_train, x_validation, y_validation)
        result.append((n,clf_accuracy,tt_time))
    return result

In [85]:
%%time
trigram_result = classifier(n_features=100000,ngram_range=(1,3))

Validation result for Logistic Regression
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
null accuracy: 50.75%
accuracy score: 82.30%
model is 31.55% more accurate than null accuracy
train and test time: 294.09s
--------------------------------------------------------------------------------
Validation result for Linear SVC
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)
null accuracy: 50.75%
accuracy score: 81.96%
model is 31.20% more accurate than null accuracy
train and test time: 314.30s
-------------------------------------------

In [86]:
print ("Train set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_train),
                                                                             (len(x_train[y_train == 0]) / (len(x_train)*1.))*100,
                                                                            (len(x_train[y_train == 1]) / (len(x_train)*1.))*100))
print ("Validation set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_validation),
                                                                             (len(x_validation[y_validation == 0]) / (len(x_validation)*1.))*100,
                                                                            (len(x_validation[y_validation == 1]) / (len(x_validation)*1.))*100))
print ("Test set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_test),
                                                                             (len(x_test[y_test == 0]) / (len(x_test)*1.))*100,
                                                                            (len(x_test[y_test == 1]) / (len(x_test)*1.))*100))

Train set has total 1547039 entries with 49.93% negative, 50.07% positive
Validation set has total 15786 entries with 50.75% negative, 49.25% positive
Test set has total 15787 entries with 50.22% negative, 49.78% positive


In [88]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils

In [92]:
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result

In [133]:
#Using logistic regression and tfidf vectorizer

pipeline2 = Pipeline([
    ('classifier', lr)
])

BSBefore = pd.read_csv("bs1cleanB.csv")
lr_fit = pipeline2.fit(x1, y)


In [96]:
BSBefore.head()

,TweetId,CleanText,timestamp,likes,replies,retweets,username
0,1183895217759576065,harvey weinstein reportedly helped clinton cam...,2019-10-14 23:59:50,0,0,0,manfretti123
1,1183895200093093888,heard folks petebuttigieg nt want pocket chang...,2019-10-14 23:59:46,0,0,0,AlabasterSquare
2,1183895128508915717,bernie sanders would raise corporate tax rate ...,2019-10-14 23:59:29,0,0,0,TT_Investors
3,1183894916855939074,harvey weinstein reportedly helped clinton cam...,2019-10-14 23:58:38,0,0,0,JackieMcReath1
4,1183894898392666112,bernie sanders calls amazon hometown tax battl...,2019-10-14 23:58:34,0,0,0,JoziBoySA


In [108]:
x2 = tvec.fit_transform(BSBefore['CleanText'].values.astype('U'))


prediction = lr_fit.predict(x2)

In [113]:
prediction

array([1, 0, 0, ..., 1, 1, 1])

In [114]:
np.mean(prediction)

0.615333078101072

In [116]:
AKBefore = pd.read_csv("ak2cleanB.csv")
AKBefore.head()


In [121]:
len(AKBefore)

1376

In [136]:
tvec = TfidfVectorizer(stop_words = None, max_features = 100000, ngram_range = (1,3))

tfidf_transformer = tvec.fit(Sentiment.SentimentText)
ak_x = tfidf_transformer.transform(AKBefore['CleanText'].values.astype('U'))

ak_x = tvec.fit_transform(AKBefore['CleanText'].values.astype('U'))
AKprediction = lr_fit.predict(ak_x) 

In [156]:
AKprediction[1:20]


array([0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1])

In [157]:
AKBefore.CleanText[1:20]

1     dem candidates congress gates bezos agreed don...
2     asked pay thedemocrats answer trump mexico ame...
3     troll tweets amy klobuchar adam schiff eric sw...
4     tonight demdebate democratic presidential cand...
5     want real president vote amy klobuchar quid pr...
6     maybe speakerpelosi tell candidates drop since...
7     hoping amyklobuchar far better candidate polls...
8     hi everyone name amy klobuchar running preside...
9     lindseygrahamsc silent photo marie amyklobucha...
10    current dems ranking solid warren wavering ewa...
11    candidates qualified tonight demdebate former ...
12                                   go amyklobuchar go
13    watch democratic debate bernie sanders elizabe...
14    update amyklobuchar also joining us tomorrow m...
15                             still like amy klobuchar
16    honestly amyklobuchar oneliners make think may...
17    know top get press hope ashleyrparker mitchell...
18    kamalaharris joebiden amyklobuchar drop su

In [139]:
AK_BeforeScore = np.mean(AKprediction)
AK_BeforeScore

0.45421511627906974

In [146]:
AYBefore = pd.read_csv("ay2cleanB.csv")
AYBefore.head()

,TweetId,CleanText,timestamp,likes,replies,retweets,username
0,1197243090605690881,cnn msnbc go inadvertently forgot mention andr...,2019-11-20 19:59:31,2,0,1,Restorer_Healer
1,1197243020535570433,andrew least speaking time every time efficien...,2019-11-20 19:59:14,0,0,0,Shigga11
2,1197243012604055552,unfortanly already started,2019-11-20 19:59:12,0,1,0,yanggang101
3,1197242987903717376,watch democratic debate andrew yang support an...,2019-11-20 19:59:06,1,0,0,TheAverageLefty
4,1197242902721773568,andrew yang promises give americans usd month ...,2019-11-20 19:58:46,1,0,0,reallyiscnn


In [175]:
tvec = TfidfVectorizer(stop_words = None, max_features = 100000, ngram_range = (1,3))
tfidf_transformer = tvec.fit(Sentiment.SentimentText)

ak_x = tvec.fit_transform(AKBefore['CleanText'].values.astype('U'))
AKprediction = lr_fit.predict(ak_x)

In [145]:
np.mean(AYprediction)

0.45421511627906974